<img src="Images/LL3-backdrop.png">

### Dates/Deadlines

Code: <i>29/04/2019 (one week before submission)</i><br />
Report: <i>05/05/2019 (20:00)</i><br />

### Imports

 You'll need to install the following libraries:
 * Gym (should have it by now)
 * Box2D
  * brew install swig
  * pip install box2d-py
  * pip install box2d


### Questions

 * Should we do [LunarLanderContinuous-v2](https://gym.openai.com/envs/LunarLanderContinuous-v2/) or [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/)?


### Comments
 
 * I think we should start with the discrete [LunarLander](https://gym.openai.com/envs/LunarLander-v2/) first then if we have the time attempt the continous one.
 
### Report

The project report should describe the problem you address, present your approach (or
approaches, if you experimented with more than one approach), and evaluate how well you have
solved the problem. It should also discuss alternative solution methods that are applicable, along
with the relative merits of your approach. Finally, the report should describe your personal
experience with the project, for example, the difficulties or any pleasant surprises you have
encountered along the way. Do not explain material that has been discussed in the lectures. For
example, there is no need to explain Q-learning or function approximation. 

Page limit: 8

### TO-DO

 * Create the agent
 * Save videos after $n$ iterations
 * Look into including the Neural Network like Mar.io (not important) 
 * We need to think of some "alternative solution" to this problem

 
### Links

 * [GitHub](https://github.bath.ac.uk/jwm59/RL-Robotics-Project)
 * [Trouble install Box2D - helped me](https://github.com/openai/gym/issues/100)
 * [More trouble shooting with Box2D](http://www.jtdz-solenoids.com/stackoverflow_/questions/44198228/install-pybox2d-for-python-3-6-with-conda-4-3-21)


In [1]:
#Importing
import gym
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

from IPython.display import clear_output, display #Not sure what this is doing exactly

import Box2D #Not sure we need this, check later


#####
import gym
import numpy as np
from IPython.display import clear_output, display
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random
import copy
import matplotlib.pyplot as plt


Using TensorFlow backend.


### Setting up the gym environment

## LunarLander-v2

* Observation Space: 8
 * X
 * Y
 * X velocity
 * Y velocity
 * Self angle
 * Angular velocity
 * Left leg contact 
 * Right lef contact
* Action Space: 4
 * Do nothing
 * Fire left orientation engine
 * Fire right orientation engine
 * Fire main engine


## LunarLanderContinuous-v2

* Observation Space: 8
 * X
 * Y
 * X velocity
 * Y velocity
 * Self angle
 * Angular velocity
 * Left leg contact 
 * Right lef contact
* Action Space: two real values vector from -1 to +1
 * First controls main engine, -1..0 off, 0..+1 throttle from 50% to 100% power
 * Second value -1.0..-0.5 fire left engine, +0.5..+1.0 fire right engine, -0.5..0.5 off


In [2]:
env_discrete = gym.make('LunarLander-v2')
env_continuous = gym.make('LunarLanderContinuous-v2')

# Deep Q-learning

### Approach

"Probably should explain the approach we plan to take..."

In [3]:
## Creating Deep-Q network
def q_network(numInput, numHidden, numHiddenLayers, numOutput, optimiserFunction=tf.train.AdamOptimizer, \
              alpha=0.001, lossFunction="mse", hiddenActivation="relu", outputActivation="linear"): 
    
    #Creating a TensorFlow class
    network = Sequential()
    
    #Creating first hidden layer
    network.add(Dense(numHidden, input_dim=numInput, activation=hiddenActivation))
    
    #Adding 'n' hidden layers
    for _ in range(numHiddenLayers):
        network.add(Dense(numHidden, activation=hiddenActivation))
        
    #Creating output layer
    network.add(Dense(numOutput, activation=outputActivation))
    
    #Defining the loss function, the optimiser and the metrics.
    network.compile(loss=lossFunction, optimizer=Adam(lr=alpha))
    
    return network   

In [4]:
### Full agent class. Look back on the w
class rocketMan_V2():
    def __init__(self, environment, numObservations, numActions, numHidden, numHiddenLayer, modelFileName, \
                     gamma=0.99, alpha=0.0001, epsilon=1, epsilonDecay=0.995, epsilon_min=0.01, tau=0.95, \
                     load=False, save=True):

        self.numObservations = numObservations
        self.numActions = numActions
        self.numHidden = numHidden
        self.numHiddenLayer = numHiddenLayer
        
        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = epsilon
        self.tau = tau
        
        self.epsilon = epsilon
        self.epsilonDecay = epsilonDecay
        self.epsilon_min = epsilon_min
        
        #replay buffer
        self.replay_buffer = []
        
        # The networks
        self.q_network = q_network(self.numObservations,self.numHidden,self.numHiddenLayer,self.numActions)
        self.q_target_network = q_network(self.numObservations,self.numHidden,self.numHiddenLayer,self.numActions)
                
        if(load):
            self.q_network.load_weights(modelFileName)
        
        self.soft_target_weight_update(1)
        
        ##### HISTORY
        self.deck = deque(maxlen=2000)
        
    def updateEpsilon(self):
        '''
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilonDecay
        
        if(self.epsilon < self.epsilon_min):
            self.epsilon = self.epsilon_min
        '''
        self.epsilon = max(self.epsilon_min, self.epsilonDecay*self.epsilon)
    
    def select_epsilon_greedy_action(self, state):            
        if np.random.uniform(0, 1) < self.epsilon:
            return(env.action_space.sample())  # random action
        else:
            q_values = self.q_network.predict(state)
            return(np.argmax(q_values[0]))        # greedy action

    def soft_target_weight_update(self, tau):
        # Q network weights
        weights = np.asarray(self.q_network.get_weights())
        
        # Target network weights
        target_weights = np.asarray(self.q_target_network.get_weights())
        
        # 
        self.q_target_network.set_weights( (weights * tau) + (target_weights * (1 - tau)))

    def learn_from_transition(self, action, state,next_state, reward, done):
        '''
        q_values = self.q_network.predict(state)

        target_q_values = q_values
        
        if done == True: 
            target_q_values[0][action] = reward
        else: 
            target_q_values[0][action] = reward + (self.gamma * np.max(self.q_target_network.predict(next_state)) )
        
        self.q_network.fit(state, target_q_values, epochs=8, batch_size=32, verbose=0)
        '''
        target = reward
        if not done: #((1-ALPHA)*xreward)+ (ALPHA* (GAMMA * futurereward))
            target = ( (1.0-0.1)*reward + 0.1 * (self.gamma*np.amax(self.q_network.predict(next_state)[0])))                

        target_old = self.q_network.predict(state)
        target_old[0][action] = target
        # Train
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_‌​parallelism_threads=‌​32, inter_op_parallelism_threads=32)))
        self.q_network.fit(x=state, y=target_old,\
                            #batch_size=1,\
                            verbose=0,\
                            epochs=1)

    def learn_from_m_random_transitions_in_replay_buffer(self, batch=16):
        '''
        assert m >= 1
        
        size_of_replay_buffer = len(self.replay_buffer)
        
        sortTest = sorted(self.replay_buffer, reverse = True, key=lambda tup: tup[3])
        
        for _ in range(m):                          # m is number of transitions
            transition_index = random.randint(0, size_of_replay_buffer-1)         
            action, state, next_state, reward, done_bool = self.replay_buffer[-(transition_index+1)]
            self.learn_from_transition(action, state, next_state, reward, done_bool)    
        '''
        sample_indx = random.sample(self.deck, batch)
        
        for action, current_state, next_state, reward, done in sample_indx: 
            self.learn_from_transition(action, current_state, next_state, reward, done)
            
        self.soft_target_weight_update(0.5)    
        
    def add_to_replay_buffer(self, action, current_state, next_state, reward, done): 
        transition_tuple = (action, current_state, next_state, reward, done)
        self.replay_buffer.append(transition_tuple)
        
        self.deck.append(transition_tuple)

### Training - Basic

Before looking into Experience Replay and Target Networks, I thought it would be best to implement a basic model first.

In [5]:
#https://github.com/RMiftakhov/LunarLander-v2-drlnd

env = env_discrete

numObservation = env.observation_space.shape[0]
numActions = env.action_space.n
numHidden = 128
numHiddenLayer = 1 #Already has a hidden layer

modelFileName = "LunarLanderWeights.h5"

agent = rocketMan_V2(env, numObservation, numActions, numHidden, numHiddenLayer, modelFileName,
                    save=True, load=False)
starting_state = env.reset() # Getting things ready

num_episodes = 1000
max_steps = 1000

reward_array = np.ndarray(shape = (num_episodes, max_steps))
steps_array = np.ndarray(shape = (num_episodes))

rewardList = []
meanRewards = []
meanRewards1 = []
episodeList = []
scores_window = deque(maxlen=100)
scores_window1 = deque(maxlen=20)
for episode in range(num_episodes):
    step = 1
    reward_sum = 0
    
    # reseting the environment and getting the starting episode action. 
    current_state = env.reset().reshape(1, numObservation)
    chosen_action = agent.select_epsilon_greedy_action(current_state)    
    
# The main loop for a given episode ------------------------------
    done = False
    #while done == False and step < max_steps:   # The problem is here:
    while done == False:
            
        next_state, reward, done, info = env.step(chosen_action)
        next_state = next_state.reshape(1, numObservation)  # Tedious reshaping needed.
        
        agent.add_to_replay_buffer(chosen_action, current_state, next_state, reward, done)
        
        # Setting things up for the next iteration of the while loop 
        current_state = next_state
        chosen_action = agent.select_epsilon_greedy_action(current_state)
        
        step += 1
        reward_sum += reward
        
        if(episode == num_episodes-1):
            env.render()
            
        if(reward_sum < -300):
            done = True
            
    steps_array[episode] = step    
    
    # We
    agent.learn_from_m_random_transitions_in_replay_buffer(16)
    rewardList.append(reward_sum)
    episodeList.append(agent.epsilon)
    scores_window.append(reward_sum)
    scores_window1.append(reward_sum)
    #agent.epsilon *= agent.epsilonDecay
    agent.updateEpsilon()
    
    clear_output(wait=True)
    
    #if(episode > 20):
    #    meanRewards.append(np.mean(rewardList[-19:]))
    meanRewards.append(np.mean(scores_window))
    meanRewards1.append(np.mean(scores_window1))

    plt.figure(figsize=(20, 10))
    ax = plt.subplot(2,2,1)
    #ax.plot(range(episode+1), rewardList)
    ax2 = ax.twinx()
    ax.plot(range(episode+1), rewardList, 'g-')
    ax2.plot(range(episode+1), episodeList, 'b-')

    ax = plt.subplot(2,2,2)
    ax.plot(range(len(meanRewards)), meanRewards, label="100 mean")
    ax.plot(range(len(meanRewards1)), meanRewards1, label="20 mean")
    plt.legend()
    plt.show()
    
    
    #Saving the trained model
agent.q_network.save_weights(modelFileName)
env.close()

plt.plot(range(num_episodes), rewardList)
plt.show()



KeyboardInterrupt: 